In [ ]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)
blue = (43, 156, 212)
red = (212, 100, 100)
orange = (249, 182, 118)
green = (43, 212, 156)

def constrain(val, min_val, max_val):
    return min(max(val, min_val), max_val)

In [ ]:
# paths = ['dice1.jpg', 'dice2.jpg', 'dice4.jpg']
# paths = ['dice2.jpg']
paths = ['ten-3.png']

In [ ]:
results = []
for i, path in enumerate(paths):
    sub_results = {}

    img = cv2.imread('../imgs/{}'.format(path))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sub_results['01-gray'] = img_gray

    blurred = cv2.blur(img_gray, (20, 20))
    sub_results['02-blurred'] = blurred

    # white dice on black
#     retval, threshold = cv2.threshold(blurred, 150, 255, cv2.THRESH_BINARY)
    # black dice on white
    retval, threshold = cv2.threshold(img_gray, 70, 255, cv2.THRESH_BINARY_INV)
    sub_results['03-threshold'] = threshold
    
    
    min_idx = img.shape.index(min(img.shape[0:2]))
    max_idx = abs(1 - min_idx)
    ratio = img.shape[min_idx] / img.shape[max_idx]
    dim = 200
    size = (dim, int(dim * ratio))
    resized = cv2.resize(threshold, size)
#     eroded = cv2.erode(resized, (1, 1), iterations = 1)
    eroded = resized
    sub_results['04-eroded'] = eroded

    eroded, contours, hierarchy = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    color = cv2.cvtColor(eroded, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(color, contours, -1, orange, 1, cv2.LINE_AA)

    for cidx, contour in enumerate(contours):
        # dice min area (rotated rect)
        rect_min_area = cv2.minAreaRect(contour)
        rect_min_points = cv2.boxPoints(rect_min_area)
        rect_min_box = np.int0(rect_min_points)
        cv2.drawContours(color, [rect_min_box], 0, red, 1, cv2.LINE_AA)

        # bounding rect of the *min area rect*
        rrb = cv2.boundingRect(rect_min_points)
        rrb_tl = rrb[0:2]
        rrb_br = tuple([sum(x) for x in zip(rrb_tl, rrb[2:4])])
        rrb_center = (rrb_tl[0] + rrb[2] / 2, rrb_tl[1] + rrb[3] / 2)
        cv2.rectangle(color, rrb_tl, rrb_br, blue, 1, cv2.LINE_AA)
        
        sub_results['05-rects'] = color
        
        cropped = eroded[rrb_tl[1]:rrb_br[1], rrb_tl[0]:rrb_br[0]]
        sub_results['06-cropped'] = cropped
        
        angle = rect_min_area[2]
        keep = angle > -45.
        
        rrb_width = rrb_br[0] - rrb_tl[0]
        rrb_height = rrb_br[1] - rrb_tl[1]
        width = rrb_width if keep else rrb_height
        height = rrb_height if keep else rrb_width
        angle += (0 if keep else 90)
        center = (width / 2, height / 2)

        dsize = (width, height)
        matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            
        straight = cv2.warpAffine(cropped, matrix, dsize)
            
        sub_results['07-straight'] = straight
        
        rect_min_translated = [(p[0] - rrb_center[0], p[1] - rrb_center[1]) for p in rect_min_points]
        rect_min_translated_box = np.int0(rect_min_translated)
        cv2.drawContours(color, [rect_min_translated_box], 0, green, 1, cv2.LINE_AA)
        
        radians = (-angle / 180) * np.pi
        c = matrix[0][0]
        s = matrix[1][0]
        rect_min_rotated = [[p[0] * c - p[1] * s, p[1] * c + p[0] * s] for p in rect_min_translated]
        rect_min_rotated_box = np.int0(rect_min_rotated)
        cv2.drawContours(color, [rect_min_rotated_box], 0, orange, 1, cv2.LINE_AA)
        
        rect_min_straight = [(p[0] + rrb_center[0], p[1] + rrb_center[1]) for p in rect_min_rotated]
        rect_min_straight_box = np.int0(rect_min_straight)
        cv2.drawContours(color, [rect_min_straight_box], 0, orange, 1, cv2.LINE_AA)
        
        rms_cropped = [[p[0] - rrb_tl[0], p[1] - rrb_tl[1]] for p in rect_min_straight]
        rms_cropped_box = np.int0(rms_cropped)
        
        nonzero = straight.nonzero()
        
        y_start = min(nonzero[0])
        y_end = max(nonzero[0])
        
        x_start = min(nonzero[1])
        x_end = max(nonzero[1])
                
        straight_crop = straight[y_start:y_end, x_start:x_end]    
        sub_results['08-final'] = straight_crop
            
        s = straight_crop.shape[0:2]
        max_dim = max(s)
        min_dim = min(s)
        x_start = int((max_dim - s[0]) / 2)
        x_end = x_start + s[0]
        y_start = int((max_dim - s[1]) / 2)
        y_end = y_start + s[1]
    
        square = np.zeros((max_dim, max_dim))
        square[x_start:x_end, y_start:y_end] = straight_crop
        
        small = cv2.resize(square, (40, 40))
        filename = 'output/{}-{}.png'.format(os.path.splitext(path)[0], str(cidx).zfill(2))
        cv2.imwrite(filename, small)
        print('wrote to {}'.format(filename))
        
    results.append(sub_results)


In [ ]:
# print(results[0].keys())
plt.rcParams['figure.figsize'] = (50, 50)
fig = plt.figure()
for result in results:
    c = 1
    for k, v in result.items():
        a = fig.add_subplot(5, 2, c)
        c += 1
        plt.imshow(v, 'gray')
        a.set_title(k)
    